# Importing necessary libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from itertools import cycle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from sklearn.utils import shuffle

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1, l2

# Creating the dataframe

In [ ]:
def read_csv_create_train_test(test_split):
    """
    test_split (input): The percentage of data that will be used to test
    """
    # Specify the path to your csv files
    csv_files = glob.glob('trial/*.csv')

    # List comprehension to load all csv files into dataframe
    dataframes = [pd.read_csv(file) for file in csv_files]
    
    # Concatenate all dataframes into one
    combined_dataframe = pd.concat(dataframes, ignore_index=True)

    # Shuffle the dataset to ensure a good mix of data points
    combined_dataframe = shuffle(combined_dataframe, random_state=42)

    # Remove sample count
    combined_dataframe.drop(['Sample Count'], axis=1, inplace=True)

    X = combined_dataframe.drop(['Movement', axis=1])
    y = combined_dataframe.Movement

    # Convert labels to one-hot encoding
    y = to_categorical(y)

    # Splitting the dataset into the Training set and Testing set. Stratify should keep all classes of movement balanced
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_split, random_state=42, stratify=y)

    # Feature scaling because we don't want one indepedent variable dominating the other and it makes computation easy
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)

    return combined_dataframe, X_train, X_test, y_train, y_test

In [ ]:
df, X_train, X_test, y_train, y_test = read_csv_create_train_test(test_split=0.3)
df

# Training the ANN model

In [ ]:
def build_model():
    # Initializing the ANN
    classifier = Sequential()

    # Adding the input layer
    classifier.add(Dense(units=X_train.shape[1], activation='relu', input_dim=X_train.shape[1], kernel_regularizer=l1(0.01)))

    # Adding the second hidden layer
    classifier.add(Dense(units=128, activation='relu', kernel_regularizer=l2(0.01)))

    # Adding the output layer
    classifier.add(Dense(units=4, activation='softmax'))

    # Compiling the ANN
    adam = Adam(learning_rate=0.001)
    classifier.compile(optimizer=adam, loss='categorical_crossentropy', metricxs=['accuracy'])

    return classifier

In [ ]:
model = build_model()

# Create and EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Training the ANN on the training set
model.fit(X_train, y_train, validation_split=0.2, batch_size=32, epochs=500, callbacks=[early_stopping])

# Predicting the test set results
y_pred = model.predict(X_test)

# Evaluate the model
_, accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy*100:.2f}%")

# Creating a confusion matrix to visualize how accurate the model is

In [ ]:
# Convert probabilities to class labels
y_pred_classes = np.argmax(y_pred, axis=1)

# Convert one-hot encoded y_test to class labels
y_test_classes = np.argmax(y_test, axis=1)

# Making the confusion matrix
cm = confusion_matrix(y_test_classes, y_pred_classes)
print(cm)

# Plot the confusion matrix
p = sns.heatmap(pd.Dataframe(cm), annot=True, cmap="YlGnBu", fmt='g')
plt.title("Confusion matrix", y=1.1)
plt.ylabel("Actual label")
plt.xlabel("Predicted label")

# Clasification report

In [ ]:
print(classification_report(y_test_classes, y_pred_classes))

In [ ]:
y_pred_proba = model.predict(X_test)

# Compute ROC curve and ROC area for each class
n_classes = y_test.shape[1]
frp = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_pred_proba[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plotting the ROC curce
colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'green', 'red'])
for i, color in zip(rage(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
    label='ROC curve of class {0} (area = {1:0.2f})'
    ''.format(i, roc_auc[i]))

plt.plot([0,1], [0,1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='lower right')
plt.show()

# Exporting the model

In [ ]:
model.save('model.h5')

In [ ]:
import tensorflow as tf

# Load the Keras model
keras_model = tf.keras.models.load_model('model.h5')

# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
tflite_model = converter.convert()

# Save the TFLite model
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)